In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

/Users/konstantin/opt/anaconda3/envs/conda_env/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train_data = pd.read_csv('/Users/konstantin/MachineLearning/task4/data/train.csv')
test_data = pd.read_csv('/Users/konstantin/MachineLearning/task4/data/test.csv')
#delete ids
train_data = train_data.drop(columns=["Id"])
val_ids = test_data["Id"]  #remember for submission
test_data = test_data.drop(columns=["Id"])
# delete nans
for feature in train_data.columns:
    percent = train_data[feature].isnull().sum() / train_data.shape[0]
    if percent > 0.7:
        train_data = train_data.drop(columns=feature)
        test_data = test_data.drop(columns=feature)
# need to convert data
for feature in train_data.columns[:-1]:
    if train_data[feature].dtype == 'object':
        train_data[feature] = LabelEncoder().fit_transform(train_data[feature])
        test_data[feature] = LabelEncoder().fit_transform(test_data[feature])
# split
X_train, X_test, y_train, y_test = train_test_split(train_data.drop(columns='SalePrice').values,
                                                    np.log1p(train_data['SalePrice'].values), test_size=0.2,
                                                    random_state=98987)
X_train = SimpleImputer(strategy='most_frequent').fit_transform(X_train)
X_test = SimpleImputer(strategy='most_frequent').fit_transform(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train, y_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    "learning_rate": [0.1, 0.2, 0.3],
    "max_depth": [2, 3, 4, 5, 6],
    "min_child_weight": [1, 2, 3, 4, 5],
    "n_estimators": [100, 300, 600, 1000]}
xgb_model = GridSearchCV(xgb.XGBRegressor(), parameters)
xgb_model.fit(X_train, y_train)
print("Best parameters for XGB is: {}".format(xgb_model.best_params_))

In [ ]:
xgb_model = xgb.XGBRegressor(**xgb_model.best_params_)
xgb_model.fit(X_train, y_train)
print('AbsError train:', metrics.mean_absolute_error(xgb_model.predict(X_train), y_train)) # сильный оверфитинг
print('AbsError test:', metrics.mean_absolute_error(xgb_model.predict(X_test), y_test))